In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import numpy as np
import os
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

First thing first we need to define some of the constraints of the problem.

First off there are only 2 classes (either cat and dog).

We can use the sigmoid activation function since we only have two classes

The pictures are in color (RGB)

We also have two folders one for cats and the other for dogs.

This means when we make the full dataset, we will have to turn all the pictures into arrays. ensure we keep the labels.
Then we need to make sure that all the cats and dogs datasets can be combined and that you choose the data randomly (with test_train_split)



In [3]:
#I will begin by turning the images into numpy arrays

cat_pics_path = r'C:\Users\franc\Documents\neural_nets\resnet_cats_vs_dogs\PetImages\Cat'
dog_pics_path = r'C:\Users\franc\Documents\neural_nets\resnet_cats_vs_dogs\PetImages\Dog'

cat_image_arrays = []
dog_image_arrays = []



In [4]:
for cat_pic in os.listdir(cat_pics_path):
    if cat_pic.lower().endswith(('.png', '.jpg', '.jpeg')):
        filepath = os.path.join(cat_pics_path, cat_pic)
        try:
            #open the image using pillow
            img = Image.open(filepath)
            #convert image to numpy array
            cats_np_array = np.array(img)
            #append the array to list
            cat_image_arrays.append(cats_np_array)
        except Exception as e:
            print(f"Error loading image {filepath}: {e}")



Error loading image C:\Users\franc\Documents\neural_nets\resnet_cats_vs_dogs\PetImages\Cat\666.jpg: cannot identify image file 'C:\\Users\\franc\\Documents\\neural_nets\\resnet_cats_vs_dogs\\PetImages\\Cat\\666.jpg'


In [5]:
len(cat_image_arrays)

12499

In [6]:
for dog_pic in os.listdir(dog_pics_path):
    if dog_pic.lower().endswith(('.png','.jpg','.jpeg')):
        filepath = os.path.join(dog_pics_path,dog_pic)
        try:
            #open the image using pillow
            img = Image.open(filepath)
            #convert image to numpy array
            dog_pic_array = np.array(img)
            #append the array to the list
            dog_image_arrays.append(dog_pic_array)
        except Exception as e:
            print(f"Error loading image {filepath}: {e}")

Error loading image C:\Users\franc\Documents\neural_nets\resnet_cats_vs_dogs\PetImages\Dog\11702.jpg: cannot identify image file 'C:\\Users\\franc\\Documents\\neural_nets\\resnet_cats_vs_dogs\\PetImages\\Dog\\11702.jpg'


c:\Users\franc\Documents\neural_nets\neural_nets\Lib\site-packages\PIL\TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [7]:
len(dog_image_arrays)

12499

In [22]:
print(type(dog_image_arrays))
print(len(dog_image_arrays))
print(type(dog_image_arrays[0]))
print(dog_image_arrays[0].shape)
print(type(dog_image_arrays[0][0]))
print(dog_image_arrays[0][0].shape)
print(type(dog_image_arrays[0][0][0]))
print(dog_image_arrays[0][0][0].shape)

<class 'list'>
12499
<class 'numpy.ndarray'>
(375, 500, 3)
<class 'numpy.ndarray'>
(500, 3)
<class 'numpy.ndarray'>
(3,)


In [33]:
cat_image_arrays[0]

array([[[203, 164,  87],
        [203, 164,  87],
        [204, 165,  88],
        ...,
        [240, 201, 122],
        [239, 200, 121],
        [238, 199, 120]],

       [[203, 164,  87],
        [203, 164,  87],
        [204, 165,  88],
        ...,
        [240, 201, 122],
        [239, 200, 121],
        [239, 200, 121]],

       [[203, 164,  87],
        [203, 164,  87],
        [204, 165,  88],
        ...,
        [241, 202, 123],
        [240, 201, 122],
        [239, 200, 121]],

       ...,

       [[153, 122,  55],
        [153, 122,  55],
        [153, 122,  55],
        ...,
        [  2,   2,   0],
        [  2,   2,   0],
        [  2,   2,   0]],

       [[152, 121,  54],
        [152, 121,  54],
        [152, 121,  54],
        ...,
        [  1,   1,   0],
        [  1,   1,   0],
        [  1,   1,   0]],

       [[151, 120,  53],
        [151, 120,  53],
        [152, 121,  54],
        ...,
        [  1,   1,   0],
        [  1,   1,   0],
        [  1,   1,   0]]

In [21]:
cat_image_arrays[0][0]

array([[203, 164,  87],
       [203, 164,  87],
       [204, 165,  88],
       ...,
       [240, 201, 122],
       [239, 200, 121],
       [238, 199, 120]], shape=(500, 3), dtype=uint8)

In [30]:
cat_image_arrays[0][0][0]

array([203, 164,  87], dtype=uint8)

In [ ]:
cat_df = pd.DataFrame({'image_array': cat_image_arrays, 'label': '0'})

cat_df.head()

,image_array,label
0,"[[[203, 164, 87], [203, 164, 87], [204, 165, 8...",0
1,"[[[36, 41, 37], [37, 42, 38], [38, 42, 41], [3...",0
2,"[[[25, 29, 38], [23, 27, 36], [19, 23, 32], [1...",0
3,"[[[223, 224, 219], [223, 224, 219], [223, 224,...",0
4,"[[[133, 107, 70], [137, 109, 70], [143, 114, 7...",0


In [ ]:
cat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12499 entries, 0 to 12498
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_array  12499 non-null  object
 1   label        12499 non-null  object
dtypes: object(2)
memory usage: 195.4+ KB


I will now make the train and validation split. 

Kindly remember that this will be an image pipeline not a dataframe pipeline since turning the data into an df pipeline would flatten the data

In [ ]:

IMG_SIZE = (224,224)
BATCH =32
SEED = 42
DATA_DIR = r'C:\Users\franc\Documents\neural_nets\resnet_cats_vs_dogs\PetImages'

train_ds = keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    label_mode='int',
    class_names=['Cat','Dog']
    labels = "inferred"
)

val_ds = keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    label_mode='int',
    class_names=['Cat','Dog']
    labels = "inferred"
)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

I then want to do some augmentation on the train part of the data. 

So that the finetuning can be more effective

In [ ]:
from tensorflow.keras import layers as L

data_augment = tf.keras.Sequential([
    L.RandomFlip("horizontal"),
    L.RandomRotation(0.05),        # ~±9°
    L.RandomZoom(0.1),
    L.RandomContrast(0.1),
])

# Apply during training only:
augmented_train_ds = train_ds.map(lambda x, y: (data_augment(x, training=True), y))
# augmented_train_ds = augmented_train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

Importing the Resnet50 model

The reason why I chose Resnet50 is because it can be imported through tensorflow unlike some other smaller variants of resnet that can only be used with the transformers library.

In [ ]:
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

I will first attempt to use the frozen model to make predictions using the pictures that we have